Code to clean-up songs without artist (Used it for any artist marked as Release)
Note: Only takes in a single column csv

In [31]:
import pandas as pd
import requests

# Replace with your actual LastFM API key
API_KEY = '4937a83f2b86cc56c8b8ee24e242081e'
API_URL = 'http://ws.audioscrobbler.com/2.0/'

# Load the CSV file
df = pd.read_csv('songs.csv')  # Ensure this file exists in your working directory

# Function to fetch artist name from LastFM
def get_artist(song_name):
    params = {
        'method': 'track.search',
        'track': song_name,
        'api_key': API_KEY,
        'format': 'json',
        'limit': 1
    }
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        try:
            return data['results']['trackmatches']['track'][0]['artist']
           ## print('hi')
        except (IndexError, KeyError):
            return 'Unknown'
    else:
        return 'Error'
print(get_artist("A Soul Can't Be Cut"),get_artist("butter"))

df['artist'] = df['song'].apply(get_artist)

df.to_csv("songs_with_artists.csv")



Kit Walters BTS


KeyError: 'song'

Combining the output with the CSV containing SongID and Song

In [ ]:

import pandas as pd

# Load the two CSV files
df1 = pd.read_csv('songs.csv')  # Contains 'songid' and 'song'
df2 = pd.read_csv('songs_with_artists.csv', names=['artist'])  # Contains only 'artist'

# Combine them side-by-side
combined_df = pd.concat([df1, df2], axis=1)

# Save to a new CSV
combined_df.to_csv('combined_output.csv', index=False)

print("Combined CSV saved as 'combined_output.csv'")


Getting Artist MetaData, namely artist type and country
Warning: Takes a while, 2s delay after each row

In [8]:
import pandas as pd
import requests
import time

# Load your CSV
df = pd.read_csv('new_missing.csv')  # Make sure this has 'songid' and 'artist' columns

# MusicBrainz API base URL
MB_API = 'https://musicbrainz.org/ws/2/artist/'

# Headers to identify your app (required by MusicBrainz)
HEADERS = {
    'User-Agent': 'Music App (mgarnier@live.ca)'
}

# Function to query MusicBrainz
def get_artist_info(artist_name):
    params = {
        'query': artist_name,
        'fmt': 'json',
        'limit': 1
    }
    try:
        response = requests.get(MB_API, headers=HEADERS, params=params)
        if response.status_code == 200:
            data = response.json()
            if data['artists']:
                artist = data['artists'][0]
                return {
                    'type': artist.get('type', ''),
                    'country': artist.get('area', ''), # Change this to area to get other values
                }
    except Exception as e:
        print(f"Error fetching {artist_name}: {e}")
    return {'type': '', 'country': ''}

# Apply the function with delay to respect rate limits
artist_data = []
for name in df['artist']:
    info = get_artist_info(name)
    artist_data.append(info)
    time.sleep(2)  # Respect MusicBrainz rate limit

# Merge results
info_df = pd.DataFrame(artist_data)
result_df = pd.concat([df.reset_index(drop=True), info_df], axis=1)

# Save to new CSV
result_df.to_csv('new_missing_artists_with_info_1.csv', index=False)
print("Saved enriched artist data to 'Spotify_artists_with_info.csv'")

Error fetching _Blan Kato: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching Diversity: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching Bluedoom: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching Bourgeoisie: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching Burbank: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching Serge Fiori: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Saved enriched artis